In [76]:
import nanomotif as nm
import polars as pl
import matplotlib.pyplot as plt
import re
import numpy as np
padding = 12

# Intro

Documentation of considerations in the implementation of the candidate selection. 

The selection is based on the sequences around methylation sites on the contig. At each methylated position, a frame of n posiion on each side of the methylation side is extracted and the index within this subsequence is considered relative to the methylation position (range: -n to n or 0:n*2).

The initial approach for candidate selection is outlined here:

1. Sample random sequences in contig at canonical bases (e.g. A when 6mA i evaluated)
2. Calculate [PSSM](https://cs.rice.edu/~ogilvie/comp571/pssm/) for contig sequences
3. Extract sequences in contig at all methylation sites
4. Set motif candidate to canonical base
2. While len(methylations sequences) > minimum sequences 
    - calculate PSSM for methylations sequences
    - Calculate KL-divergence from methylation PSSM to contig PSSM
    - Select position in sequence with highest distance
    - Select most frequent base at this position
    - Add base to motif
    - Score new motif
    - if score > threshold
        - remove seuqences with motif from methylation sequences
        - keep motif and score
    - else
        - select sequences with motif from methylation sequences

This should grow a motif by incorporation the most informative positions and bases first. 

In [77]:
def motif_identification_workflow(pileup, contig, modtype, fraction_threshold, min_cov = 5, padding = 12, min_kl_divergence = 0.01, min_sequences = 500, cdf_limit = 0.55):
    pileup_meth = pileup.filter(pl.col("mod_type") == modtype).filter(pl.col("fraction_mod") > fraction_threshold) 
    methylation_index = pileup_meth \
        .filter(pl.col("strand") == "+") \
        .filter(pl.col("Nvalid_cov") > min_cov) \
        .get_column("position").to_list()
    methylation_sequences = nm.seq.EqualLengthDNASet([nm.seq.DNAsequence(contig[(i - padding):(i + padding + 1)]) for i in methylation_index])
    return nm.evaluate.identify_motifs(methylation_sequences, contig, pileup_meth, min_kl_divergence = min_kl_divergence, min_sequences = min_sequences, cdf_limit = cdf_limit)


In [78]:
assembly = nm.load_assembly("../data/ecoli/assembly.polished.fasta")
ecoli = nm.load_pileup("../data/ecoli/modkit.pileup.bed")

In [79]:
ecol_6ma = motif_identification_workflow(ecoli.pileup, assembly["contig_3"], "a", 0.8)

............A............ | cdf score: 0.000 | n seqs:    21390 | max kl: inf
...........GA............ | cdf score: 0.000 | n seqs:    20956 | max kl: 1.074
...........GAT........... | cdf score: 0.000 | n seqs:    18970 | max kl: 1.257
...........GATC.......... | cdf score: 1.000 | n seqs:    18817 | max kl: 1.243
Saving candidate
............A............ | cdf score: 0.000 | n seqs:     2604 | max kl: inf
............AC........... | cdf score: 0.000 | n seqs:     2170 | max kl: 0.300
............AC......G.... | cdf score: 0.000 | n seqs:      897 | max kl: 0.487
............AC......GT... | cdf score: 0.000 | n seqs:      645 | max kl: 0.906
............AC......GTT.. | cdf score: 0.000 | n seqs:      567 | max kl: 0.579
Too few sequences left
............A............ | cdf score: 0.000 | n seqs:     2327 | max kl: inf
............AG........... | cdf score: 0.000 | n seqs:     1893 | max kl: 0.269
............AGG.......... | cdf score: 0.000 | n seqs:      857 | max kl: 0.818
......

In [80]:
pl.DataFrame(ecol_6ma)

column_0,column_1,column_2
str,object,f64
"""...........GAT…","BetaBernoulliModel(alpha=33978, beta=506)",1.0


In [81]:
ecol_5mc = motif_identification_workflow(ecoli.pileup, assembly["contig_3"], "m", 0.8)

............C............ | cdf score: 0.000 | n seqs:    12359 | max kl: inf
............C.G.......... | cdf score: 0.000 | n seqs:    12275 | max kl: 1.353
............C.GG......... | cdf score: 0.000 | n seqs:    12023 | max kl: 1.314
...........CC.GG......... | cdf score: 1.000 | n seqs:    11979 | max kl: 1.249
Saving candidate


In [82]:
pl.DataFrame(ecol_5mc)

column_0,column_1,column_2
str,object,f64
"""...........CC.…","BetaBernoulliModel(alpha=22041, beta=16477)",1.0


# M. ruber

In [83]:
mruber_assembly = nm.load_assembly("../data/mruber/assembly.polished.fasta")
mruber = nm.load_pileup("../data/mruber/modkit.pileup.bed")

In [84]:
mruber_6ma = motif_identification_workflow(mruber.pileup, mruber_assembly["contig_1"], "a", 0.8)

............A............ | cdf score: 0.000 | n seqs:    27680 | max kl: inf
...........GA............ | cdf score: 0.000 | n seqs:    21875 | max kl: 0.449
...........GAG........... | cdf score: 0.000 | n seqs:    14845 | max kl: 0.660
.........T.GAG........... | cdf score: 0.000 | n seqs:     7574 | max kl: 1.068
........CT.GAG........... | cdf score: 1.000 | n seqs:     6499 | max kl: 0.999
Saving candidate
............A............ | cdf score: 0.000 | n seqs:    21290 | max kl: inf
............AT........... | cdf score: 0.000 | n seqs:    15485 | max kl: 0.649
...........GAT........... | cdf score: 0.000 | n seqs:    10551 | max kl: 0.577
...........GATC.......... | cdf score: 1.000 | n seqs:     7081 | max kl: 0.850
Saving candidate
............A............ | cdf score: 0.000 | n seqs:    14447 | max kl: inf
...........AA............ | cdf score: 0.000 | n seqs:     8642 | max kl: 0.304
...........AA.T.......... | cdf score: 0.000 | n seqs:     5039 | max kl: 0.592
...........A

In [85]:
pl.DataFrame(mruber_6ma).sort("column_2", descending=True).get_column("column_0").to_list()

['........CT.GAG...........',
 '...........GATC..........',
 '...........AATT..........',
 '.........GGGAGC..........',
 '.........TTAA............',
 '.........GGCA......TGG...']

In [86]:
mruber_5mc = motif_identification_workflow(mruber.pileup, mruber_assembly["contig_1"], "m", 0.8)

............G............ | cdf score: 0.000 | n seqs:     1250 | max kl: inf
............GA........... | cdf score: 0.000 | n seqs:      511 | max kl: 0.479
Too few sequences left
............C............ | cdf score: 0.000 | n seqs:      926 | max kl: inf
Too few sequences left


In [87]:
mruber_5mc

([], [], [])

# Geobacillus

In [88]:
geobacillus_assembly = nm.load_assembly("../data/geobacillus/assembly.polished.fasta")
geobacillus = nm.load_pileup("../data/geobacillus/modkit.pileup.bed")

In [89]:
geobacillus_assembly.assembly

{'contig_3': DNAsequence() | Number of sequence: 82915 | Unique Bases: ['A', 'G', 'C', 'T'],
 'contig_2': DNAsequence() | Number of sequence: 93311 | Unique Bases: ['A', 'G', 'C', 'T'],
 'contig_1': DNAsequence() | Number of sequence: 3857718 | Unique Bases: ['A', 'G', 'C', 'T']}

In [94]:
geobacillus_6ma = motif_identification_workflow(geobacillus.pileup.filter(pl.col("contig") == "contig_1"), geobacillus_assembly["contig_1"], "a", 0.8, min_sequences=200)

............A............ | cdf score: 0.000 | n seqs:    15963 | max kl: inf
............A.C.......... | cdf score: 0.000 | n seqs:    15756 | max kl: 0.853
...........GA.C.......... | cdf score: 0.000 | n seqs:    13195 | max kl: 1.066
...........GATC.......... | cdf score: 1.000 | n seqs:    11617 | max kl: 1.218
Saving candidate
............A............ | cdf score: 0.000 | n seqs:     4350 | max kl: inf
...........AA............ | cdf score: 0.000 | n seqs:     4143 | max kl: 0.497
...........AAG........... | cdf score: 0.000 | n seqs:     3065 | max kl: 0.755
.......G...AAG........... | cdf score: 0.000 | n seqs:     1556 | max kl: 1.468
.......G..GAAG........... | cdf score: 0.000 | n seqs:     1552 | max kl: 1.407
.......G..GAAGC.......... | cdf score: 0.000 | n seqs:     1551 | max kl: 0.763
.......GA.GAAGC.......... | cdf score: 1.000 | n seqs:      904 | max kl: 0.706
Saving candidate
............A............ | cdf score: 0.000 | n seqs:     3714 | max kl: inf
.........C..

In [95]:
geobacillus_6ma

(['...........GATC..........',
  '.......GA.GAAGC..........',
  '........CCAAAT...........',
  '........ACCCA............',
  '.......GA.GAAGT..........',
  '.......GG.GAAGC..........',
  '.......GG.GAAGT..........'],
 [BetaBernoulliModel(alpha=21448, beta=3052),
  BetaBernoulliModel(alpha=1255, beta=160),
  BetaBernoulliModel(alpha=2847, beta=321),
  BetaBernoulliModel(alpha=1804, beta=778),
  BetaBernoulliModel(alpha=872, beta=197),
  BetaBernoulliModel(alpha=534, beta=109),
  BetaBernoulliModel(alpha=396, beta=135)],
 [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

In [96]:
geobacillus_5mc = motif_identification_workflow(geobacillus.pileup.filter(pl.col("contig") == "contig_1"), geobacillus_assembly["contig_1"], "m", 0.5)

............C............ | cdf score: 0.000 | n seqs:     1411 | max kl: inf
............C...C........ | cdf score: 0.000 | n seqs:     1314 | max kl: 0.482
............CG..C........ | cdf score: 0.000 | n seqs:      873 | max kl: 0.981
............CG.TC........ | cdf score: 0.000 | n seqs:      700 | max kl: 1.331
............CGATC........ | cdf score: 0.000 | n seqs:      694 | max kl: 1.186
...........CCGATC........ | cdf score: 0.000 | n seqs:      693 | max kl: 0.654
Too few sequences left
............C............ | cdf score: 0.000 | n seqs:      928 | max kl: inf
...........GC............ | cdf score: 0.000 | n seqs:      831 | max kl: 0.300
Too few sequences left
............C............ | cdf score: 0.000 | n seqs:      588 | max kl: inf
Too few sequences left


In [97]:
geobacillus_5mc

([], [], [])